# Library

In [ ]:
!pip install PySastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk import word_tokenize,sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load Data

In [ ]:
from google.colab import drive

In [ ]:
df = pd.read_csv("Dataprepoo.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69040 entries, 0 to 69039
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   full_text  68120 non-null  object 
 1   sentimen   69026 non-null  float64
 2   keyword    69040 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1.6+ MB


In [ ]:
df.columns

Index(['full_text', 'sentimen', 'keyword'], dtype='object')

In [ ]:
df.head(20)

,full_text,sentimen,keyword
0,1646820629033910000 @Sukardi2017 Sekarang kita...,-1.0,anies
1,1646820850451220000 @Anak__Ogi @ganjarpranowo ...,0.0,anies
2,1646821153460360000 @Relawananies Ya justru da...,-1.0,anies
3,"1646821761907050000 """"@yusufagro @prabowo @ben...",1.0,anies
4,1646822542647390000 @Stev3n_Peg3l Banyak bengk...,-1.0,anies
5,"1646822594660950000 """"@Quvvatt @gtobing2903 @a...",-1.0,anies
6,1646822810650640000 Wajib jadi presiden? Silah...,-1.0,anies
7,1646823169226060000 PP Darul Ulum mendukung @a...,1.0,anies
8,1646823233000450000 @langitan99 satu kubu @ani...,0.0,anies
9,"1646824381031790000 """"DPP NasDem Gelar Buka Pu...",1.0,anies


In [ ]:
df['sentimen'].value_counts()

 1.0     27286
 0.0     24681
-1.0     17058
 11.0        1
Name: sentimen, dtype: int64

In [ ]:
if (df['sentimen'].isna().sum() != 0):
  df.dropna(subset= ["sentimen"],inplace = True)
else:
    pass
df['sentimen'].isna().sum()

0

In [ ]:
#check duplicate

df.duplicated(subset= ['full_text']).sum()

3918

In [ ]:
df.drop_duplicates(subset=['full_text'], inplace=True)
df.reset_index(inplace=True)
len(df)

65108

In [ ]:
df['sentimen'].value_counts()

 1.0    24813
 0.0    23763
-1.0    16532
Name: sentimen, dtype: int64

In [ ]:
df['sentimen'] = df['sentimen'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65108 entries, 0 to 65107
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      65108 non-null  int64 
 1   full_text  65107 non-null  object
 2   sentimen   65108 non-null  int64 
 3   keyword    65108 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.0+ MB


In [ ]:
X = df['full_text']
y = df['sentimen']

In [ ]:
X

0        1646820629033910000 @Sukardi2017 Sekarang kita...
1        1646820850451220000 @Anak__Ogi @ganjarpranowo ...
2        1646821153460360000 @Relawananies Ya justru da...
3        1646821761907050000 ""@yusufagro @prabowo @ben...
4        1646822542647390000 @Stev3n_Peg3l Banyak bengk...
                               ...                        
65103    @PandamPamungkas @ganjarpranowo @Rosianna766Hi...
65104    Komunitas Disabilitas Tuna Rungu Indonesia men...
65105    Komunitas Disabilitas Tuna Rungu Indonesia men...
65106    Calon Ibu Negara gw nih gaess ðŸ¤© Sdh mempuny...
65107    @Nikmatul_Sg @ganjarpranowo Alam Ganjar adalah...
Name: full_text, Length: 65108, dtype: object

In [ ]:
y

0       -1
1        0
2       -1
3        1
4       -1
        ..
65103    1
65104    1
65105    1
65106    1
65107    1
Name: sentimen, Length: 65108, dtype: int64

In [ ]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{-1: 16532, 0: 23763, 1: 24813}

In [ ]:
df['sentimen'].value_counts()

 1    24813
 0    23763
-1    16532
Name: sentimen, dtype: int64

In [ ]:
df.dropna(subset=["full_text"], inplace=True)

In [ ]:
df['sentimen'].value_counts()

 1    24813
 0    23763
-1    16531
Name: sentimen, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65107 entries, 0 to 65107
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      65107 non-null  int64 
 1   full_text  65107 non-null  object
 2   sentimen   65107 non-null  int64 
 3   keyword    65107 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.5+ MB


# Functions

In [ ]:
def hapus_tanda(full_text):
    tanda_baca = set(string.punctuation)
    full_text = ''.join(ch for ch in full_text if ch not in tanda_baca)
    return full_text

def hapus_katadouble(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def hapus_imbuhan(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(text)

def cleaning(full_text):
    #remove url
    full_text = re.sub(r"http\S+", "", full_text)
    #remove hex characters
    full_text = re.sub(r'(\\x(.){2})', '', full_text)
    #remove newlines
    full_text = re.sub(r'\s+|\\n', ' ', full_text)
    #remove Retweet (RT) in a tweet
    full_text = re.sub(r'RT ','', full_text)
    #remove mention
    full_text = re.sub(r"@[A-Za-z0-9_]+","", full_text)
    #remove hashtags
    full_text = re.sub(r'\B#\w*[a-zA-Z]+\w*','', full_text)
    #remove single char
    full_text = re.sub(r'\b[a-zA-Z]\b','', full_text)
    #remove number
    full_text = re.sub(r'\d+','', full_text)
    # full_text = re.sub(r'E+','', full_text)
    #remove dollar sign (untuk data saham dari stockbit)
    full_text = re.sub(r'\$.{4}', '',full_text)
    #remove punctuation / tanda baca
    full_text = re.sub(r'[^\w\s]+', ' ', full_text)
    #remove spaces at the beginning of tweet
    full_text = re.sub(r"/^\\s+/m", '', full_text)
    #remove multiplechar
    # full_text = re.sub(r'(.)\1+', r'\1', full_text)
    regex = r'\b(\w+)(?:\W+\1\b)+'
    full_text = re.sub(r"\n", " ",full_text)
    full_text = re.sub(r"\r", " ",full_text)
    full_text = re.sub(r"&amp;", " ",full_text)
    full_text = re.sub(r"&gt;", " ",full_text)
    full_text = re.sub(r"\d"," ",full_text)
    full_text = re.sub(r"[_]",' ',full_text)
    # hapus #tagger
    full_text = re.sub(r'#([^\s]+)', r'\1', full_text)
    #hapus hastag
    full_text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", full_text).split())
    # hapus URL
    full_text = re.sub(r"[-_<>*#$@()\"/%&;:<>{}`+=~|.!?,']", " ", full_text)
    # Menghapus karakter aneh
    full_text = re.sub(r'[^A-Za-z0-9\s]', '', full_text)
    # hapus simbol
    full_text = re.sub(r'''(?i)\b((?:https|http?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "",full_text)    # hapus simbol
    # hapus ASCII
    # hapus double spasi
    full_text = re.sub("\s\s+", " ",full_text)
    # hapus tanda baca
    full_text = hapus_tanda(full_text)
    # hapus angka dan angka yang berada dalam string
    full_text = re.sub(r'\w*\d\w*', '',full_text).strip()
    # hapus repetisi karakter
    full_text = hapus_katadouble(full_text)
    # hapus Emoji
    emoji_pattern =  re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    full_text = emoji_pattern.sub(r'',full_text)
    # menghapus perulangan kata
    full_text = re.sub(regex, r'\1', full_text, flags=re.IGNORECASE)
    full_text = hapus_imbuhan(full_text)
    return full_text

In [ ]:
#!gdown --id '1YRHYMHz6ukjqX1R9rAYrPbx2Placy0m0'

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/gdown/cli.py", line 151, in main
    filename = download(
  File "/usr/local/lib/python3.10/dist-packages/gdown/download.py", line 203, in download
    filename_from_url = m.groups()[0]
AttributeError: 'NoneType' object has no attribute 'groups'


In [ ]:
kamus_alay1 = pd.read_csv('https://raw.githubusercontent.com/fendiirfan/Kamus-Alay/main/Kamus-Alay.csv')
kamus_alay1 = kamus_alay1.set_index('kataAlay')


In [ ]:
custom_word = ['nya', 'tolong', 'iya', 'guna', 'kasih', 'buka', 'hari', 'sih', 'mohon', 'baru', 'ovo', 'dana', 'linkaja', 'saku', 'sakuku', 'aplikasi',
               'idpfst', 'xianzhou', 'tk', 'play station', 'mg', 'apksi', 'xvi', 'ng', 'ipn', 'xoxpopuli', 'ipo',
               'ammn', 'cnbc', 'publica', 'liputancom', 'populi', 'vox', 'dpw', 'radarbantencoid', 'lsn', 'lasso',
               'cuador', 'guillermo', 'mov', 'indomart', 'ddy', 'psht', 'pan', 'nassundala', 'aiu', 'rpan', 'ssp',
               'smrc', 'mnc', 'cinderama', 'wkwkkwkk', 'ydh', 'blass', 'hasshh', 'zonder', 'youtubu', 'mile',
               'phakphum', 'hhhahaa', 'dgpbanten', 'tkrpp', 'wawakkwkskkeke', 'knpi', 'dk', 'wewewew', 'huushh',
               'wujajajjajajaakkakakakkakak', 'erickthohirnews', 'ilc', ]

In [ ]:
stopWord = list(pd.read_csv('https://raw.githubusercontent.com/datascienceid/stopwords-bahasa-indonesia/master/stopwords_id_satya.txt', header = None)[0])
stopWord1 = list(pd.read_csv('https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt', header = None)[0])
import requests

# Replace the URLs with the correct raw content URLs
url1 = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-id/master/raw/indonesian-stopwords-complete.txt'
url2 = 'https://raw.githubusercontent.com/TakayukiP/Stopword-Indonesia/main/stopword.txt'

# Function to fetch and read text from URL
def read_txt_from_url(url):
    response = requests.get(url)
    text = response.text
    return text

# Read the stopword lists into lists
stopWord2 = read_txt_from_url(url1).split('\n')
stopWord3 = read_txt_from_url(url2).split('\n')

# Remove empty strings or whitespaces
stopWord2 = [word.strip() for word in stopWord2 if word.strip()]
stopWord3 = [word.strip() for word in stopWord3 if word.strip()]


In [ ]:
data_kbbi = [kamus.strip('\n') for kamus in open('kamwus.tsv',encoding='utf-8')]
def change_word(text):
    kamus = {}
    for i in data_kbbi:
        if '\t' in i:
            key, val = i.split('\t', 1)  # Split only at the first tab
            kamus[str(key)] = val
        else:
            # Handle the case where there is no tab in the line
            # You may want to decide what to do in this case (skip the line, log a warning, etc.)
            pass

    final_string = ' '.join(str(kamus.get(word, word)) for word in text).split()
    return final_string


In [ ]:
data_kbbi

['15%\tlima belas persen',
 '100%\tseratus persen',
 '1000%\tseribu persen ',
 '80\tdelapan puluh',
 '2024\tdua ribu dua puluh empat ',
 '2024\tdua ribu dua empat',
 '44374\tdua puluh tujuh juni',
 ' Awokwokwokwok\thaha',
 ' dach\tdah',
 '@\tdi',
 '_kepada\tkepada',
 '_klompok\tkelompok',
 '10jt\tsepuluh juta',
 '10k\tsepuluh ribu',
 '10kk\tsepuluh juta',
 '10rb\tsepuluh ribu',
 '10th\tsepuluh tahun',
 '120k\tseratus dua puluh ribu',
 '13rb\ttiga belas ribu',
 '145k\tseratus empat puluh lima ribu',
 '149rb\tseratus empat puluh sembilan ribu',
 '14rb\tempat belas ribu',
 '15jt\tlima belas juta',
 '16gb\tenam belas giga byte',
 '16jam\tenam belas jam',
 '16th\tenam belas tahun',
 '17an\ttujuh belasan',
 '1jt\tsatu juta',
 '1kg\tsatu kilogram',
 '1periode\tsatu periode',
 '1st\tpertama',
 '1t\tsatu triliun',
 '200an\tdua ratusan',
 '200rbu\tdua ratus ribu',
 '2018gantigubernur\t2018 ganti gubernur',
 '2019asalbukanjokowi\t2019 asal bukan jokowi',
 '2019forjkw\t2019 for jokowi',
 '2019g\t2

In [ ]:
df_meaningless = pd.read_csv('meaningless.csv', header=None, encoding='latin-1')

df_meaningless.head(20)

,0
0,kamu
1,aduh
2,ah
3,ahelah
4,amp
5,aw
6,aww
7,beb
8,bebb
9,bete


In [ ]:
custom_delete_string = ['moch', 'eksan', 'apelo', 'kerajaan', 'ni', 'cakap', 'takda', 'ambik', 'kuala', 'xnak', 'nk', 'klia', 'elok', 'pulak', 'tgk', 'tengok',
                            'amek', 'atok', 'amik', 'nak', 'amp', 'tetibe', 'ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut',
 'berikutnya',
 'berjumlah',
 'berkali-kali',
 'berkata',
 'berkehendak',
 'berkeinginan',
 'berkenaan',
 'berlainan',
 'berlalu',
 'berlangsung',
 'berlebihan',
 'bermacam',
 'bermacam-macam',
 'bermaksud',
 'bermula',
 'bersama',
 'bersama-sama',
 'bersiap',
 'bersiap-siap',
 'bertanya',
 'bertanya-tanya',
 'berturut',
 'berturut-turut',
 'bertutur',
 'berujar',
 'berupa',
 'besar',
 'betul',
 'betulkah',
 'biasa',
 'biasanya',
 'bila',
 'bilakah',
 'bisa',
 'bisakah',
 'boleh',
 'bolehkah',
 'bolehlah',
 'buat',
 'bukan',
 'bukankah',
 'bukanlah',
 'bukannya',
 'bulan',
 'bung',
 'cara',
 'caranya',
 'cukup',
 'cukupkah',
 'cukuplah',
 'cuma',
 'dahulu',
 'dalam',
 'dan',
 'dapat',
 'dari',
 'daripada',
 'datang',
 'dekat',
 'demi',
 'demikian',
 'demikianlah',
 'dengan',
 'depan',
 'di',
 'dia',
 'diakhiri',
 'diakhirinya',
 'dialah',
 'diantara',
 'diantaranya',
 'diberi',
 'diberikan',
 'diberikannya',
 'dibuat',
 'dibuatnya',
 'didapat',
 'didatangkan',
 'digunakan',
 'diibaratkan',
 'diibaratkannya',
 'diingat',
 'diingatkan',
 'diinginkan',
 'dijawab',
 'dijelaskan',
 'dijelaskannya',
 'dikarenakan',
 'dikatakan',
 'dikatakannya',
 'dikerjakan',
 'diketahui',
 'diketahuinya',
 'dikira',
 'dilakukan',
 'dilalui',
 'dilihat',
 'dimaksud',
 'dimaksudkan',
 'dimaksudkannya',
 'dimaksudnya',
 'diminta',
 'dimintai',
 'dimisalkan',
 'dimulai',
 'dimulailah',
 'dimulainya',
 'dimungkinkan',
 'dini',
 'dipastikan',
 'diperbuat',
 'diperbuatnya',
 'dipergunakan',
 'diperkirakan',
 'diperlihatkan',
 'diperlukan',
 'diperlukannya',
 'dipersoalkan',
 'dipertanyakan',
 'dipunyai',
 'diri',
 'dirinya',
 'disampaikan',
 'disebut',
 'disebutkan',
 'disebutkannya',
 'disini',
 'disinilah',
 'ditambahkan',
 'ditandaskan',
 'ditanya',
 'ditanyai',
 'ditanyakan',
 'ditegaskan',
 'ditujukan',
 'ditunjuk',
 'ditunjuki',
 'ditunjukkan',
 'ditunjukkannya',
 'ditunjuknya',
 'dituturkan',
 'dituturkannya',
 'diucapkan',
 'diucapkannya',
 'diungkapkan',
 'dong',
 'dua',
 'dulu',
 'empat',
 'enggak',
 'enggaknya',
 'entah',
 'entahlah',
 'guna',
 'gunakan',
 'hal',
 'hampir',
 'hanya',
 'hanyalah',
 'hari',
 'harus',
 'haruslah',
 'harusnya',
 'hendak',
 'hendaklah',
 'hendaknya',
 'hingga',
 'ia',
 'ialah',
 'ibarat',
 'ibaratkan',
 'ibaratnya',
 'ibu',
 'ikut',
 'ingat',
 'ingat-ingat',
 'ingin',
 'inginkah',
 'inginkan',
 'ini',
 'inikah',
 'inilah',
 'itu',
 'itukah',
 'itulah',
 'jadi',
 'jadilah',
 'jadinya',
 'jangan',
 'jangankan',
 'janganlah',
 'jauh',
 'jawab',
 'jawaban',
 'jawabnya',
 'jelas',
 'jelaskan',
 'jelaslah',
 'jelasnya',
 'jika',
 'jikalau',
 'juga',
 'jumlah',
 'jumlahnya',
 'justru',
 'kala',
 'kalau',
 'kalaulah',
 'kalaupun',
 'kalian',
 'kami',
 'kamilah',
 'kamu',
 'kamulah',
 'kan',
 'kapan',
 'kapankah',
 'kapanpun',
 'karena',
 'karenanya',
 'kasus',
 'kata',
 'katakan',
 'katakanlah',
 'katanya',
 'ke',
 'keadaan',
 'kebetulan',
 'kecil',
 'kedua',
 'keduanya',
 'keinginan',
 'kelamaan',
 'kelihatan',
 'kelihatannya',
 'kelima',
 'keluar',
 'kembali',
 'kemudian',
 'kemungkinan',
 'kemungkinannya',
 'kenapa',
 'kepada',
 'kepadanya',
 'kesampaian',
 'keseluruhan',
 'keseluruhannya',
 'keterlaluan',
 'ketika',
 'khususnya',
 'kini',
 'kinilah',
 'kira',
 'kira-kira',
 'kiranya',
 'kita',
 'kitalah',
 'kok',
 'kurang',
 'lagi',
 'lagian',
 'lah',
 'lain',
 'lainnya',
 'lalu',
 'lama',
 'lamanya',
 'lanjut',
 'lanjutnya',
 'lebih',
 'lewat',
 'lima',
 'luar',
 'macam',
 'maka',
 'makanya',
 'makin',
 'malah',
 'malahan',
 'mampu',
 'mampukah',
 'mana',
 'manakala',
 'manalagi',
 'masa',
 'masalah',
 'masalahnya',
 'masih',
 'masihkah',
 'masing',
 'masing-masing',
 'mau',
 'maupun',
 'melainkan',
 'melakukan',
 'melalui',
 'melihat',
 'melihatnya',
 'memang',
 'memastikan',
 'memberi',
 'memberikan',
 'membuat',
 'memerlukan',
 'memihak',
 'meminta',
 'memintakan',
 'memisalkan',
 'memperbuat',
 'mempergunakan',
 'memperkirakan',
 'memperlihatkan',
 'mempersiapkan',
 'mempersoalkan',
 'mempertanyakan',
 'mempunyai',
 'memulai',
 'memungkinkan',
 'menaiki',
 'menambahkan',
 'menandaskan',
 'menanti',
 'menanti-nanti',
 'menantikan',
 'menanya',
 'menanyai',
 'menanyakan',
 'mendapat',
 'mendapatkan',
 'mendatang',
 'mendatangi',
 'mendatangkan',
 'menegaskan',
 'mengakhiri',
 'mengapa',
 'mengatakan',
 'mengatakannya',
 'mengenai',
 'mengerjakan',
 'mengetahui',
 'menggunakan',
 'menghendaki',
 'mengibaratkan',
 'mengibaratkannya',
 'mengingat',
 'mengingatkan',
 'menginginkan',
 'mengira',
 'mengucapkan',
 'mengucapkannya',
 'mengungkapkan',
 'menjadi',
 'menjawab',
 'menjelaskan',
 'menuju',
 'menunjuk',
 'menunjuki',
 'menunjukkan',
 'menunjuknya',
 'menurut',
 'menuturkan',
 'menyampaikan',
 'menyangkut',
 'menyatakan',
 'menyebutkan',
 'menyeluruh',
 'menyiapkan',
 'merasa',
 'mereka',
 'merekalah',
 'merupakan',
 'meski',
 'meskipun',
 'meyakini',
 'meyakinkan',
 'minta',
 'mirip',
 'misal',
 'misalkan',
 'misalnya',
 'mula',
 'mulai',
 'mulailah',
 'mulanya',
 'mungkin',
 'mungkinkah',
 'nah',
 'naik',
 'namun',
 'nanti',
 'nantinya',
 'nyaris',
 'nyatanya',
 'oleh',
 'olehnya',
 'pada',
 'padahal',
 'padanya',
 'pak',
 'paling',
 'panjang',
 'pantas',
 'para',
 'pasti',
 'pastilah',
 'penting',
 'pentingnya',
 'per',
 'percuma',
 'perlu',
 'perlukah',
 'perlunya',
 'pernah',
 'persoalan',
 'pertama',
 'pertama-tama',
 'pertanyaan',
 'pertanyakan',
 'pihak',
 'pihaknya',
 'pukul',
 'pula',
 'pun',
 'punya',
 'rasa',
 'rasanya',
 'rata',
 'rupanya',
 'saat',
 'saatnya',
 'saja',
 'sajalah',
 'saling',
 'sama',
 'sama-sama',
 'sambil',
 'sampai',
 'sampai-sampai',
 'sampaikan',
 'sana',
 'sangat',
 'sangatlah',
 'satu',
 'saya',
 'sayalah',
 'se',
 'sebab',
 'sebabnya',
 'sebagai',
 'sebagaimana',
 'sebagainya',
 'sebagian',
 'sebaik',
 'sebaik-baiknya',
 'sebaiknya',
 'sebaliknya',
 'sebanyak',
 'sebegini',
 'sebegitu',
 'sebelum',
 'sebelumnya',
 'sebenarnya',
 'seberapa',
 'sebesar',
 'sebetulnya',
 'sebisanya',
 'sebuah',
 'sebut',
 'sebutlah',
 'sebutnya',
 'secara',
 'secukupnya',
 'sedang',
 'sedangkan',
 'sedemikian',
 'sedikit',
 'sedikitnya',
 'seenaknya',
 'segala',
 'segalanya',
 'segera',
 'seharusnya',
 'sehingga',
 'seingat',
 'sejak',
 'sejauh',
 'sejenak',
 'sejumlah',
 'sekadar',
 'sekadarnya',
 'sekali',
 'sekali-kali',
 'sekalian',
 'sekaligus',
 'sekalipun',
 'sekarang',
 'sekarang',
 'sekecil',
 'seketika',
 'sekiranya',
 'sekitar',
 'sekitarnya',
 'sekurang-kurangnya',
 'sekurangnya',
 'sela',
 'selain',
 'selaku',
 'selalu',
 'selama',
 'selama-lamanya',
 'selamanya',
 'selanjutnya',
 'seluruh',
 'seluruhnya',
 'semacam',
 'semakin',
 'semampu',
 'semampunya',
 'semasa',
 'semasih',
 'semata',
 'semata-mata',
 'semaunya',
 'sementara',
 'semisal',
 'semisalnya',
 'sempat',
 'semua',
 'semuanya',
 'semula',
 'sendiri',
 'sendirian',
 'sendirinya',
 'seolah',
 'seolah-olah',
 'seorang',
 'sepanjang',
 'sepantasnya',
 'sepantasnyalah',
 'seperlunya',
 'seperti',
 'sepertinya',
 'sepihak',
 'sering',
 'seringnya',
 'serta',
 'serupa',
 'sesaat',
 'sesama',
 'sesampai',
 'sesegera',
 'sesekali',
 'seseorang',
 'sesuatu',
 'sesuatunya',
 'sesudah',
 'sesudahnya',
 'setelah',
 'setempat',
 'setengah',
 'seterusnya',
 'setiap',
 'setiba',
 'setibanya',
 'setidak-tidaknya',
 'setidaknya',
 'setinggi',
 'seusai',
 'sewaktu',
 'siap',
 'siapa',
 'siapakah',
 'siapapun',
 'sini',
 'sinilah',
 'soal',
 'soalnya',
 'suatu',
 'sudah',
 'sudahkah',
 'sudahlah',
 'supaya',
 'tadi',
 'tadinya',
 'tahu',
 'tahun',
 'tak',
 'tambah',
 'tambahnya',
 'tampak',
 'tampaknya',
 'tandas',
 'tandasnya',
 'tanpa',
 'tanya',
 'tanyakan',
 'tanyanya',
 'tapi',
 'tegas',
 'tegasnya',
 'telah',
 'tempat',
 'tengah',
 'tentang',
 'tentu',
 'tentulah',
 'tentunya',
 'tepat',
 'terakhir',
 'terasa',
 'terbanyak',
 'terdahulu',
 'terdapat',
 'terdiri',
 'terhadap',
 'terhadapnya',
 'teringat',
 'teringat-ingat',
 'terjadi',
 'terjadilah',
 'terjadinya',
 'terkira',
 'terlalu',
 'terlebih',
 'terlihat',
 'termasuk',
 'ternyata',
 'tersampaikan',
 'tersebut',
 'tersebutlah',
 'tertentu',
 'tertuju',
 'terus',
 'terutama',
 'tetap',
 'tetapi',
 'tiap',
 'tiba',
 'tiba-tiba',
 'tidak',
 'tidakkah',
 'tidaklah',
 'tiga',
 'tinggi',
 'toh',
 'tunjuk',
 'turut',
 'tutur',
 'tuturnya',
 'ucap',
 'ucapnya',
 'ujar',
 'ujarnya',
 'umum',
 'umumnya',
 'ungkap',
 'ungkapnya',
 'untuk',
 'usah',
 'usai',
 'waduh',
 'wah',
 'wahai',
 'waktu',
 'waktunya',
 'walau',
 'walaupun',
 'wong',
 'yaitu',
 'yakin',
 'yakni',
 'yang', 'yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 'kalo', 'amp',
                     'biar', 'bikin', 'bilang', 'krn', 'nya', 'nih', 'sih', 'si',
                     'tau', 'tdk', 'tuh', 'utk', 'ya', 'jd', 'jgn', 'sdh', 'aja',
                     'n', 't', 'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                     '&amp', 'p', 'jd', 'nek', 'e', 'yo', 'o', 'np', 'nw',
                     'https', 'http', 't', 'co', 'moilnyaw', 'agshshsjs','dmp',
                     'dfess','zdy','ahiak','wooooy', 'hhahahahahaha','murmuring',
                     'hhaaddeehhhh','eey','raheuuut','hemmmhh','ahahahahahahaha',
                     'bunggg','ndag','urghhhh','xdd','uehe','ewkwkwk','ltl',
                     'aaaakk','rvlog','cuuy','broooooooooo','woiiiiiii','woooooh',
                     'yeuuu','hahahhaahhaahahhah','dihh','yass','utbk','skskskk',
                     'hahahahahhaha','brrip', 'humn','laaaaah','hikd','wkkwkwkwk',
                     'hhahahaah','hahahahahaah','wwkaokwoakwoak','awkakwkawkak',
                     'weyyy','umphhh','pfttttttt','lsc','nyoih','dhhddgh',
                     'uuiii','tbz','hahhhhh','bhak','tteh','nyingnying','pfff',
                     'aiiiihhh','thg','akakakakak','pgw','bradd','wessss','dgm',
                     'miiin','diihh','ajskahsjajsjajsk','woilah','tars','cokkk',
                     'jiaahhhh','wkwkwkwkwkwkwkwk','wkwkwkkwkw','coeg','upis',
                     'jdksbskebsksj','hahehoh','uwuwuwuwuuw', 'aksjsjsksks','jcm',
                     'ndeeeer','hwhw','ququ','boooook','kwah','nyees','geekbahpodcast',
                     'sumargo','flickmagazine','lacuk','wowoowowowowowoo','fassbender',
                     'aaaaak','uwaaaaa','yaelahhh','djdiidhdhsndndjdijdd','fsai',
                     'rrrrrah','aaaaakkk','eeeuuuuhhh','tenenbaum','hadohh','potus',
                     'monek','arggghhhhhh','aaaaaarghh', 'heheheheheueu','iteotfw',
                     'semsem','tobiojersey','idot','morim','sekkk','ejenan',
                     'afkzkunuqkfkigieowkdbjosk','fvck','wkakakka','terrrrrr',
                     'ajsksksjshsjs','dpmallxxi', 'yourwiuwiu','sksksjsj','peele',
                     'huuuuwwaaaahhhh','hahhart','mbahput','huwaaaaaa','fckd',
                     'mmjd','knet','gwangj','nyahahahaha','aaarhh','',
                     'booos','owo','sedeksegaraga','fxxxing','lolll','auliarani',
                     'ehehehhe','hshhshs','theeeeeeee','iqiyi','weeeeehhhh',
                     'tncfu','crotan','huancuuk', 'boxboxid','mchtsvrtn','decc',
                     'ohmy','jff','hukshuks','htgawm','ckli','huweeee',
                     'aaaaarrgghhh','antongekngok','mnet','faaak','trns'
                     'wsowksksnwk','hotrl','falk','egggghhhhhh','fakkkk',
                     'iftw','aokbab','cuklawwww','poonpiriya','nattawut',
                     'kimcop','wkwkwkwkkwkwkwkwk','ftd','ashkskaksjd',
                     'aaaaahggggg','pimchanok','satsetsatset','swadikap',
                     'bebb','hahahahahh','uwma','dangal','dpo','eyyy',
                     'provokemagazine','bhahahahak','seeeeh','wedewww',
                     'tvxq','ueueu','dinitain','lahyaa','hamberr','cexi',
                     'ngentott','wkowkwo','smlmt','fwb','ngeten','whatthefvck',
                     'diputarbelit','cakmangkok', 'ovrl','siiiihhh','daddt',
                     'paspispus','ueueueu','wkaokqowlaowkok','iwww','rdwnfrmnsyh',
                     'lapett','hddu','lsk','asshskhs','wkakakakak','wataheck',
                     'wabb','kamseeeeeeeuuuu','cooy','tefak','mzk','lautnernyaaa',
                     'uben','sils','ngekngok','foxmoviespremiumhd','pbio','tepuuu',
                     'haihhhh', 'tsom', 'wwkwkwkwkk','samoek','brayy','peroah',
                     'pppfffttt','hhahaa','hailaaa','aghf','ahfakk','jooooo',
                     'elaahhhh','kuhhh','hhhhhhhh','wowkwkwk', 'vyll','hiikkkhhh',
                     'ajskqjskaana','wooooiiiiiii','ahelah','rtya','ngebe','rctv',
                     'cuar','hunham','bocen','semprol','gamonin','upill',
                     'lyv','bamf','arghtt','hahsjdbeocnkenfkdnjsbxockskcisndkdcdknxkdmkwndkdnkandksnsnmdjdnskansjbwjsndjdnsjxndmsn',
                     'pelirit','uwoogh','dofp','deeyach','hhahahaaa','hass','mbts',
                     'akwkwk','omgomg','onggg', 'csrnya','leutualy','bebeto','shayy',
                     'widihh','aghny','skskak','wwkwkwkkw','wqwqw','wadohhh','pubg',
                     'habede','kaaah','fuckk', 'yaksip','watchmenid','akwkwoowk',
                     'eskil','klikfilm','molatv','wokwowko','luurrrr','yasss',
                     'waaaaahhhh','duuuhhhh','wooooowww','qwtwfwtwywgsvqqqqwtwuwisip',
                     'tourent', 'hfa','hshdh','tflix','waduu','duilah','drakorindo',
                     'cuiii','pft','arhdjjdb','yaaw','smzbsmale','mbaaakk','shjjshfj','ahahahhh',
                     'terwow','hajahjhs','boomm','ihiyy','snewen','skksks','shshsjsj',
                     'aksksk','aaaahsgehwnahwhwhwvwghq','hadadahhhhh','nderrrr',
                     'aksjsggs','anjahskahs','woowwwww', 'hhhhhhh','ueueueue','heyyyyy',
                     'sjsiznsksk','bahhh','hahhehhoh','shshsh','wtcb','uwahh','hshshshsh',
                     'terrrr','aaaaaak','yihuy','aaargghhh', 'oyeee','hadeeuh','ftf','dipnya',
                     'woohee','nokhorom','ihhhhh','mincot','wkwkwwkwkwkwk','bruhhhhh',
                     'akwkwow','hheu','pisuh','haaaaaaaa', 'dadadahhh','ealaaahh','asdgjhfjkgdfh',
                     'shittt','bensu','mewgulf','yeeehaaaaaa','woohoooooo','skwk','ajdhsnjdhs',
                     'asfsgahshsh','posin','koks','hueeee', 'kuaya','huwawiwew','laff','booooommm',
                     'kubaper','ngiung','awkawk','wkwkkwwk','asshdhgshsjsk','wkwkwkkwkwwkwk',
                     'blaaah','yawlaa','krktr','starbak', 'wtffff','gtv','oalahh','seeeeehhhhh',
                     'momz','mamirt','xzkx','akahdhffvh','hdhejsjdks','wle','hfft','broou',
                     'coooy','woeee', 'hfttt','sksksksks','djaaaannnnccfc','akwjwj','ooooooo',
                     'hhhfftt','kwkwkkw','cihh','laaaaaaa','areyoulostbebiguuyrl','hdeehhh','chuakss',
                     'ponhub','ovt', 'mbaaaa','oooooom','skjsw','whaha','yeyeyeee','ajdghdkh','cenya',
                     'kkekekkk','coyyyy','hahahhahahaha','ahshwvwhsbwvs','otm','arggghh','ngahaha',
                     'eeeemmmm','ahayyyyy','heyhey','tfc','ckckrt','nmn','kulapongvanich','aitthipat',
                     'bitt','kahh','aaaakkkkk','duuuu','mskwydit','wkkwkwkwkwk', 'tchy','ngkoh',
                     'eyyyyy','fahhhh','dsah','eekin','wodnrhwkxneualo','alaaaaa','ueueeee','cucmeyyy',
                     'akhkh','luwh','citraxxi','hahajja', 'hasshhh','oohalah','beeppp','jbcu','waaaawaw',
                     'ahelahhh','ffuffu','hrrrggghh','xoxooxo','arggg','wkwkkwkwkw','ajkdkdks',
                     'aksksksjdkcj','nsd', 'colimon','knetz','soop','plekkkk','heumm','ngerong',
                     'woylah','hmmzz','ahsjskskfjk','huvt','skfnfikejd','mweehehe','iddkdksk','hsjsks',
                     'masnov','lghdtv','lgbtqvwxyz','aksksnnshdksk','hshs','adoyaai','jkfhegfliagwifgiywqgfyqwgfyqgfyiegf',
                     'waaay','nintik','asdghkkl','hwhwhwh','wkakak','aowkwowk','sjsjsjsjs',
                     'aksjskajwyatsh','muahahahahah','lbhs','wkwkwkwjsjsjssj','wqwqwqwq','ftw',
                     'wiboe','tnk','ekwkwk','hshsh','awowkowk','kdpp','meehhh','ajel',
                     'beehh','mgm','angrok','yaowoohh','akakskks','ppkm','meeks','ngokkkk',
                     'akhsks','whwhwh','kstew','fipm','tgc','wasweswos', 'cretttttt','cinn',
                     'ahahabab','etdahhhh','fck','wkwkwkkwkwkwkwkwkkw','nttd','beugh','xnya',
                     'sksksks','vxkxhsnxndmdjdndjdjdn','wwkw','ngederegdeg','borr',
                     'wadau','sembet','emk','shshshshshshshshshsh','huftttttt','yayayay',
                     'aksowkdoek','jasjuss','alhambra','molotovgirl','dantdm','gituhdwndiabwdhabwfh',
                     'dctv','nyetil', 'eeaao','hshshshs','uhuuy','zzoe','atez','bler','blor','sampis',
                     'jlo','iyeel','wooowwww','arhhhg','heeuuuu','wkkwwk', 'wkakakakakaka','uhuyyy',
                     'skwmwkwmwkw','bjir','wkwkwkwkkwu','wewew','bakekok','bnha','ahik','movimax',
                     'paragonxxi','oowhhh','tsang','lgux', 'ngegarong','lahkok','tcog','wzd','jder','ntond',
                     'ginit','wadidaw','kelulus','klux','uwoh','cokk','jedak','junji', 'ahoy','nyumm',
                     'pfoa','waoww','woee','eok','acau','mcdonalds','flis','bekasinians','aaakkk',
                     'awokwok','faakkk','ueueue', 'woooy','sksks','jbjb','awikwok','slebew','bayona',
                     'nsfw','unru','tnh','kpai','jsb','ptj','snaxx','covid', 'gidik','minsik','gamon',
                     'lgbtq','animovie','tasm','jnt','fastfurious','morbius','zsjl','seeeeee','wwkwk',
                     'disneyyy','ndakik', 'imdbnya','cinecrib','wst','ifwt','cmbyn','menfess','wckd',
                     'swank','brou','fsog','ittipat','nkcthi','tabetai','giur', 'ngl','sksk', 'afuk',
                     'jcw','xtina','kroc','blcu','sksksk','trll','jpf','doss','dcu','bvs','cgv','cilers',
                     'wwy', 'tdkr','mjb', 'nwh','bcu','dceu','kdm','okja','cruella','eternals','insidous',
                     'annabel', 'insidius', 'fiftyshades', 'insidiouos']




In [ ]:
stopWord

['ada',
 'adanya',
 'adalah',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'diantaranya',
 'antara',
 'antaranya',
 'diantara',
 'apa',
 'apaan',
 'mengapa',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'atau',
 'ataukah',
 'ataupun',
 'bagai',
 'bagaikan',
 'sebagai',
 'sebagainya',
 'bagaimana',
 'bagaimanapun',
 'sebagaimana',
 'bagaimanakah',
 'bagi',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'sebaliknya',
 'banyak',
 'sebanyak',
 'beberapa',
 'seberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'sebegini',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'sebegitu',
 'belum',
 'belumlah',
 'sebelum',
 'sebelumnya',
 'sebenarnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'betulkah',
 'sebetulnya',
 'biasa',
 'biasanya',
 'bila',
 'bilakah',
 'bisa',
 'bisakah',
 'sebisanya',
 'boleh',
 'bolehkah',
 'bolehlah',
 'buat',
 'bukan',
 'bukankah',
 'bukanlah',


In [ ]:
#dari nltk
nltk_stopword = stopwords.words('indonesian')
#dari sastrawi
stopword_id_factory  = StopWordRemoverFactory()
sastrawi_stopword = stopword_id_factory.get_stop_words()
meaningless_list = df_meaningless[0].tolist()

listStopword = list(set(meaningless_list + stopWord + custom_word + stopWord1 + nltk_stopword + sastrawi_stopword + stopWord2 + stopWord3))
def stopwords_removal(words):
    return [word for word in words if word not in listStopword]

In [ ]:
print(listStopword)

['alalah', 'te', 'Ã°Å¸Å½Â¾', 'minah', 'Ã°Å¸Â\x8d', 'aditcm', 'siap', 'ataupun', 'play station', 'mocndut', 'ngalelewe', 'whatsapp', 'zannuba', 'Ã°Å¸â\x80\x99â\x80\xa0Ã°Å¸Â\x8fÂ½', 'Ã°Å¸â\x80¡Â¬Ã°Å¸â\x80¡Â§', 'sasuke', 'heheheje', 'Ã°Å¸â\x84¢â\x80¦', 'Ã¢Å\x93â\x80¹', 'ipit', 'didu', 'Ã°Å¸â\x80\x99â\x80¡Ã°Å¸Â\x8fÂ¿', 'huee', 'un', 'Ã°Å¸Å\x92â\x80\x9c', 'lfc', 'seorang', 'lillaahi', 'Ã\x82Â®', 'kalaulah', 'menggunakan', 'darmaji', 'Ã°Å¸Â\x8dÂ«', 'Ã°Å¸â\x84¢â\x80\x9a', 'Ã°Å¸â\x80\x9dÂ¬', 'Ã°Å¸Å½Â®', 'Ã°Å¸Å½Â\x81', 'heryawan', 'sesuatu', 'tarcute', 'Ã¢Å¡ Ã¯Â¸Â\x8f', 'belsonaro', 'Ã°Å¸Â\x8fÆ\x92', 'Ã¢Å\x93Å\xa0Ã°Å¸Â\x8fÂ½', 'teddy', 'haliwen', 'menunjuk', 'nohhhh', 'bayern', 'sensei', 'andikiwantono', 'hadee', 'njekethek', 'smma', 'messi', 'bu', 'aura', 'Ã°Å¸â\x80\x9cÅ\x93', 'seperti', 'keluar', 'sule', 'me', 'lho', 'Ã°Å¸â\x80\x9câ\x80°', 'dika', 'radarbantncoid', 'sekecil', 'ombudsmanri awasitegurlaporkan', 'Ã°Å¸â\x80¢Â¡', 'sedang', 'Ã°Å¸â\x80\x98Â¨Ã¢â\x82¬Â\x8dÃ°Å¸â\x80\x98Â¨Ã¢â\x82¬Â\x8dÃ

In [ ]:
def ApplyKamusAlayStopTranslate(text):
    words = text.split(' ')
    result = []

    for word in words:
        try:
            replaced = kamus.get(word, word)
            if replaced != '':
                result.append(replaced)
        except:
            pass

        try:
            mask = kamusAlay['slang'] == word
            if mask.any():
                index = kamusAlay[mask].index[0]
                formal_word = kamusAlay.at[index, 'formal']
                if formal_word != '':
                    result.append(formal_word)
        except:
            pass

        try:
            if word not in stopWord and word != '':
                result.append(word)
        except:
            pass

    return ' '.join(result)


In [ ]:
#mengembalikan token menjadi string tunggal kembali
def untokenize(text):
    text = ' '.join([t for t in text])
    return text

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    text = stemmer.stem(text)
    return text

In [ ]:
df.reset_index(drop=True,inplace = True)

In [ ]:
df.head()

,index,full_text,sentimen,keyword
0,0,1646820629033910000 @Sukardi2017 Sekarang kita...,-1,anies
1,1,1646820850451220000 @Anak__Ogi @ganjarpranowo ...,0,anies
2,2,1646821153460360000 @Relawananies Ya justru da...,-1,anies
3,3,"1646821761907050000 """"@yusufagro @prabowo @ben...",1,anies
4,4,1646822542647390000 @Stev3n_Peg3l Banyak bengk...,-1,anies


In [ ]:
df['Stemmed'] = df['full_text'].astype(str)
df['Cleaning'] = df['Stemmed'].apply(cleaning)

In [ ]:
df['Casefolding'] = df['Cleaning'].str.lower()

In [ ]:
df['Alayfix'] = df['Casefolding'].apply(ApplyKamusAlayStopTranslate)

In [ ]:
df['Alayfix']

0              viralkan aja ya tolong punya gen bohong ayo
1                     wiihh pak heru gak minat ikut ganjar
2        ya nyata yg rusak nama anies baswedan jelek or...
3                                     hidup anies baswedan
4                                           bengkak ya pak
                               ...                        
65102    bapak ganjar nunggu calon presiden republik in...
65103    komunitas disabilitas tuna rungu indonesia und...
65104    komunitas disabilitas tuna rungu indonesia aja...
65105    calon ibu negara gw nih gaess sdh punya progra...
65106      alam ganjar sosok anak calon presiden indonesia
Name: Alayfix, Length: 65107, dtype: object

In [ ]:
df['Tokenized'] = df['Alayfix'].apply(word_tokenize)

In [ ]:
df['Change Word'] = df['Tokenized'].apply(change_word)

In [ ]:
df.head(50)

,index,full_text,sentimen,keyword,Stemmed,Cleaning,Casefolding,Alayfix,Tokenized,Change Word
0,0,1646820629033910000 @Sukardi2017 Sekarang kita...,-1,anies,1646820629033910000 @Sukardi2017 Sekarang kita...,sekarang kita viralkan aja ya tolong apakah an...,sekarang kita viralkan aja ya tolong apakah an...,viralkan aja ya tolong punya gen bohong ayo,"[viralkan, aja, ya, tolong, punya, gen, bohong...","[viral, saja, iya, tolong, punya, generasi, bo..."
1,1,1646820850451220000 @Anak__Ogi @ganjarpranowo ...,0,anies,1646820850451220000 @Anak__Ogi @ganjarpranowo ...,wiihh pak heru gak minat ikut ganjar,wiihh pak heru gak minat ikut ganjar,wiihh pak heru gak minat ikut ganjar,"[wiihh, pak, heru, gak, minat, ikut, ganjar]","[wiihh, pak, heru, tidak, minat, ikut, ganjar]"
2,2,1646821153460360000 @Relawananies Ya justru da...,-1,anies,1646821153460360000 @Relawananies Ya justru da...,ya justru dari nyata ini yg rusak nama anies b...,ya justru dari nyata ini yg rusak nama anies b...,ya nyata yg rusak nama anies baswedan jelek or...,"[ya, nyata, yg, rusak, nama, anies, baswedan, ...","[iya, nyata, yang, rusak, nama, Anies, Basweda..."
3,3,"1646821761907050000 """"@yusufagro @prabowo @ben...",1,anies,"1646821761907050000 """"@yusufagro @prabowo @ben...",hidup anies baswedan,hidup anies baswedan,hidup anies baswedan,"[hidup, anies, baswedan]","[hidup, Anies, Baswedan]"
4,4,1646822542647390000 @Stev3n_Peg3l Banyak bengk...,-1,anies,1646822542647390000 @Stev3n_Peg3l Banyak bengk...,banyak bengkak ya pak,banyak bengkak ya pak,bengkak ya pak,"[bengkak, ya, pak]","[bengkak, iya, pak]"
5,5,"1646822594660950000 """"@Quvvatt @gtobing2903 @a...",-1,anies,"1646822594660950000 """"@Quvvatt @gtobing2903 @a...",udah bener itu kafir arti halang oleh tunjuk a...,udah bener itu kafir arti halang oleh tunjuk a...,udah bener kafir arti halang tunjuk allah,"[udah, bener, kafir, arti, halang, tunjuk, allah]","[sudah, benar, kafir, arti, halang, tunjuk, Al..."
6,6,1646822810650640000 Wajib jadi presiden? Silah...,-1,anies,1646822810650640000 Wajib jadi presiden? Silah...,wajib jadi presiden silah aja rakyat indonesia...,wajib jadi presiden silah aja rakyat indonesia...,wajib jadi presiden silah aja rakyat indonesia...,"[wajib, jadi, presiden, silah, aja, rakyat, in...","[wajib, jadi, presiden, saya, saja, rakyat, in..."
7,7,1646823169226060000 PP Darul Ulum mendukung @a...,1,anies,1646823169226060000 PP Darul Ulum mendukung @a...,pp darul ulum dukung beda hasil survey prof sa...,pp darul ulum dukung beda hasil survey prof sa...,pp darul ulum dukung beda hasil survey prof sa...,"[pp, darul, ulum, dukung, beda, hasil, survey,...","[pulang, pergi, darul, ulum, dukung, beda, has..."
8,8,1646823233000450000 @langitan99 satu kubu @ani...,0,anies,1646823233000450000 @langitan99 satu kubu @ani...,satu kubu,satu kubu,satu kubu,"[satu, kubu]","[satu, kubu]"
9,9,"1646824381031790000 """"DPP NasDem Gelar Buka Pu...",1,anies,"1646824381031790000 """"DPP NasDem Gelar Buka Pu...",dpp nasdem gelar buka puasa sama ratus yatim j...,dpp nasdem gelar buka puasa sama ratus yatim j...,dpp nasdem gelar buka puasa ratus yatim jabode...,"[dpp, nasdem, gelar, buka, puasa, ratus, yatim...","[dewan, pimpinan, pusat, nasional, demokrat, g..."


In [ ]:
df['Removed Stopwords'] = df['Change Word'].apply(stopwords_removal)

In [ ]:
df.head(10)

,index,full_text,sentimen,keyword,Stemmed,Cleaning,Casefolding,Alayfix,Tokenized,Change Word,Removed Stopwords
0,0,1646820629033910000 @Sukardi2017 Sekarang kita...,-1,anies,1646820629033910000 @Sukardi2017 Sekarang kita...,sekarang kita viralkan aja ya tolong apakah an...,sekarang kita viralkan aja ya tolong apakah an...,viralkan aja ya tolong punya gen bohong ayo,"[viralkan, aja, ya, tolong, punya, gen, bohong...","[viral, saja, iya, tolong, punya, generasi, bo...","[viral, generasi, bohong]"
1,1,1646820850451220000 @Anak__Ogi @ganjarpranowo ...,0,anies,1646820850451220000 @Anak__Ogi @ganjarpranowo ...,wiihh pak heru gak minat ikut ganjar,wiihh pak heru gak minat ikut ganjar,wiihh pak heru gak minat ikut ganjar,"[wiihh, pak, heru, gak, minat, ikut, ganjar]","[wiihh, pak, heru, tidak, minat, ikut, ganjar]","[wiihh, minat, ganjar]"
2,2,1646821153460360000 @Relawananies Ya justru da...,-1,anies,1646821153460360000 @Relawananies Ya justru da...,ya justru dari nyata ini yg rusak nama anies b...,ya justru dari nyata ini yg rusak nama anies b...,ya nyata yg rusak nama anies baswedan jelek or...,"[ya, nyata, yg, rusak, nama, anies, baswedan, ...","[iya, nyata, yang, rusak, nama, Anies, Basweda...","[rusak, nama, Anies, Baswedan, jelek, jelek, n..."
3,3,"1646821761907050000 """"@yusufagro @prabowo @ben...",1,anies,"1646821761907050000 """"@yusufagro @prabowo @ben...",hidup anies baswedan,hidup anies baswedan,hidup anies baswedan,"[hidup, anies, baswedan]","[hidup, Anies, Baswedan]","[hidup, Anies, Baswedan]"
4,4,1646822542647390000 @Stev3n_Peg3l Banyak bengk...,-1,anies,1646822542647390000 @Stev3n_Peg3l Banyak bengk...,banyak bengkak ya pak,banyak bengkak ya pak,bengkak ya pak,"[bengkak, ya, pak]","[bengkak, iya, pak]",[bengkak]
5,5,"1646822594660950000 """"@Quvvatt @gtobing2903 @a...",-1,anies,"1646822594660950000 """"@Quvvatt @gtobing2903 @a...",udah bener itu kafir arti halang oleh tunjuk a...,udah bener itu kafir arti halang oleh tunjuk a...,udah bener kafir arti halang tunjuk allah,"[udah, bener, kafir, arti, halang, tunjuk, allah]","[sudah, benar, kafir, arti, halang, tunjuk, Al...","[kafir, halang, Allah]"
6,6,1646822810650640000 Wajib jadi presiden? Silah...,-1,anies,1646822810650640000 Wajib jadi presiden? Silah...,wajib jadi presiden silah aja rakyat indonesia...,wajib jadi presiden silah aja rakyat indonesia...,wajib jadi presiden silah aja rakyat indonesia...,"[wajib, jadi, presiden, silah, aja, rakyat, in...","[wajib, jadi, presiden, saya, saja, rakyat, in...","[wajib, presiden, rakyat, indonesia, dukung, p..."
7,7,1646823169226060000 PP Darul Ulum mendukung @a...,1,anies,1646823169226060000 PP Darul Ulum mendukung @a...,pp darul ulum dukung beda hasil survey prof sa...,pp darul ulum dukung beda hasil survey prof sa...,pp darul ulum dukung beda hasil survey prof sa...,"[pp, darul, ulum, dukung, beda, hasil, survey,...","[pulang, pergi, darul, ulum, dukung, beda, has...","[pulang, pergi, ulum, dukung, beda, hasil, sur..."
8,8,1646823233000450000 @langitan99 satu kubu @ani...,0,anies,1646823233000450000 @langitan99 satu kubu @ani...,satu kubu,satu kubu,satu kubu,"[satu, kubu]","[satu, kubu]",[kubu]
9,9,"1646824381031790000 """"DPP NasDem Gelar Buka Pu...",1,anies,"1646824381031790000 """"DPP NasDem Gelar Buka Pu...",dpp nasdem gelar buka puasa sama ratus yatim j...,dpp nasdem gelar buka puasa sama ratus yatim j...,dpp nasdem gelar buka puasa ratus yatim jabode...,"[dpp, nasdem, gelar, buka, puasa, ratus, yatim...","[dewan, pimpinan, pusat, nasional, demokrat, g...","[dewan, pimpinan, pusat, nasional, demokrat, g..."


In [ ]:
df['Untokenized'] = df['Removed Stopwords'].apply(untokenize)

In [ ]:
df.head(10)

,index,full_text,sentimen,keyword,Stemmed,Cleaning,Casefolding,Alayfix,Tokenized,Change Word,Removed Stopwords,Untokenized
0,0,1646820629033910000 @Sukardi2017 Sekarang kita...,-1,anies,1646820629033910000 @Sukardi2017 Sekarang kita...,sekarang kita viralkan aja ya tolong apakah an...,sekarang kita viralkan aja ya tolong apakah an...,viralkan aja ya tolong punya gen bohong ayo,"[viralkan, aja, ya, tolong, punya, gen, bohong...","[viral, saja, iya, tolong, punya, generasi, bo...","[viral, generasi, bohong]",viral generasi bohong
1,1,1646820850451220000 @Anak__Ogi @ganjarpranowo ...,0,anies,1646820850451220000 @Anak__Ogi @ganjarpranowo ...,wiihh pak heru gak minat ikut ganjar,wiihh pak heru gak minat ikut ganjar,wiihh pak heru gak minat ikut ganjar,"[wiihh, pak, heru, gak, minat, ikut, ganjar]","[wiihh, pak, heru, tidak, minat, ikut, ganjar]","[wiihh, minat, ganjar]",wiihh minat ganjar
2,2,1646821153460360000 @Relawananies Ya justru da...,-1,anies,1646821153460360000 @Relawananies Ya justru da...,ya justru dari nyata ini yg rusak nama anies b...,ya justru dari nyata ini yg rusak nama anies b...,ya nyata yg rusak nama anies baswedan jelek or...,"[ya, nyata, yg, rusak, nama, anies, baswedan, ...","[iya, nyata, yang, rusak, nama, Anies, Basweda...","[rusak, nama, Anies, Baswedan, jelek, jelek, n...",rusak nama Anies Baswedan jelek jelek nama Anies
3,3,"1646821761907050000 """"@yusufagro @prabowo @ben...",1,anies,"1646821761907050000 """"@yusufagro @prabowo @ben...",hidup anies baswedan,hidup anies baswedan,hidup anies baswedan,"[hidup, anies, baswedan]","[hidup, Anies, Baswedan]","[hidup, Anies, Baswedan]",hidup Anies Baswedan
4,4,1646822542647390000 @Stev3n_Peg3l Banyak bengk...,-1,anies,1646822542647390000 @Stev3n_Peg3l Banyak bengk...,banyak bengkak ya pak,banyak bengkak ya pak,bengkak ya pak,"[bengkak, ya, pak]","[bengkak, iya, pak]",[bengkak],bengkak
5,5,"1646822594660950000 """"@Quvvatt @gtobing2903 @a...",-1,anies,"1646822594660950000 """"@Quvvatt @gtobing2903 @a...",udah bener itu kafir arti halang oleh tunjuk a...,udah bener itu kafir arti halang oleh tunjuk a...,udah bener kafir arti halang tunjuk allah,"[udah, bener, kafir, arti, halang, tunjuk, allah]","[sudah, benar, kafir, arti, halang, tunjuk, Al...","[kafir, halang, Allah]",kafir halang Allah
6,6,1646822810650640000 Wajib jadi presiden? Silah...,-1,anies,1646822810650640000 Wajib jadi presiden? Silah...,wajib jadi presiden silah aja rakyat indonesia...,wajib jadi presiden silah aja rakyat indonesia...,wajib jadi presiden silah aja rakyat indonesia...,"[wajib, jadi, presiden, silah, aja, rakyat, in...","[wajib, jadi, presiden, saya, saja, rakyat, in...","[wajib, presiden, rakyat, indonesia, dukung, p...",wajib presiden rakyat indonesia dukung preside...
7,7,1646823169226060000 PP Darul Ulum mendukung @a...,1,anies,1646823169226060000 PP Darul Ulum mendukung @a...,pp darul ulum dukung beda hasil survey prof sa...,pp darul ulum dukung beda hasil survey prof sa...,pp darul ulum dukung beda hasil survey prof sa...,"[pp, darul, ulum, dukung, beda, hasil, survey,...","[pulang, pergi, darul, ulum, dukung, beda, has...","[pulang, pergi, ulum, dukung, beda, hasil, sur...",pulang pergi ulum dukung beda hasil survei pro...
8,8,1646823233000450000 @langitan99 satu kubu @ani...,0,anies,1646823233000450000 @langitan99 satu kubu @ani...,satu kubu,satu kubu,satu kubu,"[satu, kubu]","[satu, kubu]",[kubu],kubu
9,9,"1646824381031790000 """"DPP NasDem Gelar Buka Pu...",1,anies,"1646824381031790000 """"DPP NasDem Gelar Buka Pu...",dpp nasdem gelar buka puasa sama ratus yatim j...,dpp nasdem gelar buka puasa sama ratus yatim j...,dpp nasdem gelar buka puasa ratus yatim jabode...,"[dpp, nasdem, gelar, buka, puasa, ratus, yatim...","[dewan, pimpinan, pusat, nasional, demokrat, g...","[dewan, pimpinan, pusat, nasional, demokrat, g...",dewan pimpinan pusat nasional demokrat gelar p...


In [ ]:
df[['Removed Stopwords','Untokenized']].loc[0]

Removed Stopwords    [viral, generasi, bohong]
Untokenized              viral generasi bohong
Name: 0, dtype: object

In [ ]:
df['Clean'] = df['Untokenized'].apply(stemming)

In [ ]:
empty_index = df['Clean'].loc[df['Stemmed'] == ''].index

In [ ]:
#print(len(df))
df['Stemmed'].replace('',np.nan,inplace=True)
df.dropna(subset = ['Stemmed'],inplace = True)
df.reset_index(drop=True,inplace=True)
#print(len(X_train))

In [ ]:
df['sentimen'].value_counts()

 1    24813
 0    23763
-1    16531
Name: sentimen, dtype: int64

In [ ]:
df["sentimen"] = df["sentimen"].replace([-1, 1, 0], [0, 2, 1])

In [ ]:
df['sentimen'].value_counts()

2    24813
1    23763
0    16531
Name: sentimen, dtype: int64

In [ ]:
duplicated_sum = df.duplicated('Casefolding').sum()
duplicated_index = df.loc[df.duplicated('Casefolding')].index
duplicated_index

Int64Index([   51,    90,   117,   197,   202,   215,   218,   274,   276,
              313,
            ...
            65048, 65054, 65056, 65058, 65061, 65066, 65069, 65076, 65093,
            65103],
           dtype='int64', length=5045)

In [ ]:
print('number of data:',len(df))
print('duplicated data:',duplicated_sum)

df.drop_duplicates(subset= ['Casefolding'],inplace=True)
df.reset_index(drop=True, inplace=True)

print('number of data after dropping duplicates:',len(df))

number of data: 65107
duplicated data: 5045
number of data after dropping duplicates: 60062


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head(50)

,index,full_text,sentimen,keyword,Stemmed,Cleaning,Casefolding,Alayfix,Tokenized,Change Word,Removed Stopwords,Untokenized,Clean
0,0,1646820629033910000 @Sukardi2017 Sekarang kita...,0,anies,1646820629033910000 @Sukardi2017 Sekarang kita...,sekarang kita viralkan aja ya tolong apakah an...,sekarang kita viralkan aja ya tolong apakah an...,viralkan aja ya tolong punya gen bohong ayo,"[viralkan, aja, ya, tolong, punya, gen, bohong...","[viral, saja, iya, tolong, punya, generasi, bo...","[viral, generasi, bohong]",viral generasi bohong,viral generasi bohong
1,1,1646820850451220000 @Anak__Ogi @ganjarpranowo ...,1,anies,1646820850451220000 @Anak__Ogi @ganjarpranowo ...,wiihh pak heru gak minat ikut ganjar,wiihh pak heru gak minat ikut ganjar,wiihh pak heru gak minat ikut ganjar,"[wiihh, pak, heru, gak, minat, ikut, ganjar]","[wiihh, pak, heru, tidak, minat, ikut, ganjar]","[wiihh, minat, ganjar]",wiihh minat ganjar,wiihh minat ganjar
2,2,1646821153460360000 @Relawananies Ya justru da...,0,anies,1646821153460360000 @Relawananies Ya justru da...,ya justru dari nyata ini yg rusak nama anies b...,ya justru dari nyata ini yg rusak nama anies b...,ya nyata yg rusak nama anies baswedan jelek or...,"[ya, nyata, yg, rusak, nama, anies, baswedan, ...","[iya, nyata, yang, rusak, nama, Anies, Basweda...","[rusak, nama, Anies, Baswedan, jelek, jelek, n...",rusak nama Anies Baswedan jelek jelek nama Anies,rusak nama anies baswedan jelek jelek nama anies
3,3,"1646821761907050000 """"@yusufagro @prabowo @ben...",2,anies,"1646821761907050000 """"@yusufagro @prabowo @ben...",hidup anies baswedan,hidup anies baswedan,hidup anies baswedan,"[hidup, anies, baswedan]","[hidup, Anies, Baswedan]","[hidup, Anies, Baswedan]",hidup Anies Baswedan,hidup anies baswedan
4,4,1646822542647390000 @Stev3n_Peg3l Banyak bengk...,0,anies,1646822542647390000 @Stev3n_Peg3l Banyak bengk...,banyak bengkak ya pak,banyak bengkak ya pak,bengkak ya pak,"[bengkak, ya, pak]","[bengkak, iya, pak]",[bengkak],bengkak,bengkak
5,5,"1646822594660950000 """"@Quvvatt @gtobing2903 @a...",0,anies,"1646822594660950000 """"@Quvvatt @gtobing2903 @a...",udah bener itu kafir arti halang oleh tunjuk a...,udah bener itu kafir arti halang oleh tunjuk a...,udah bener kafir arti halang tunjuk allah,"[udah, bener, kafir, arti, halang, tunjuk, allah]","[sudah, benar, kafir, arti, halang, tunjuk, Al...","[kafir, halang, Allah]",kafir halang Allah,kafir halang allah
6,6,1646822810650640000 Wajib jadi presiden? Silah...,0,anies,1646822810650640000 Wajib jadi presiden? Silah...,wajib jadi presiden silah aja rakyat indonesia...,wajib jadi presiden silah aja rakyat indonesia...,wajib jadi presiden silah aja rakyat indonesia...,"[wajib, jadi, presiden, silah, aja, rakyat, in...","[wajib, jadi, presiden, saya, saja, rakyat, in...","[wajib, presiden, rakyat, indonesia, dukung, p...",wajib presiden rakyat indonesia dukung preside...,wajib presiden rakyat indonesia dukung preside...
7,7,1646823169226060000 PP Darul Ulum mendukung @a...,2,anies,1646823169226060000 PP Darul Ulum mendukung @a...,pp darul ulum dukung beda hasil survey prof sa...,pp darul ulum dukung beda hasil survey prof sa...,pp darul ulum dukung beda hasil survey prof sa...,"[pp, darul, ulum, dukung, beda, hasil, survey,...","[pulang, pergi, darul, ulum, dukung, beda, has...","[pulang, pergi, ulum, dukung, beda, hasil, sur...",pulang pergi ulum dukung beda hasil survei pro...,pulang pergi ulum dukung beda hasil survei pro...
8,8,1646823233000450000 @langitan99 satu kubu @ani...,1,anies,1646823233000450000 @langitan99 satu kubu @ani...,satu kubu,satu kubu,satu kubu,"[satu, kubu]","[satu, kubu]",[kubu],kubu,kubu
9,9,"1646824381031790000 """"DPP NasDem Gelar Buka Pu...",2,anies,"1646824381031790000 """"DPP NasDem Gelar Buka Pu...",dpp nasdem gelar buka puasa sama ratus yatim j...,dpp nasdem gelar buka puasa sama ratus yatim j...,dpp nasdem gelar buka puasa ratus yatim jabode...,"[dpp, nasdem, gelar, buka, puasa, ratus, yatim...","[dewan, pimpinan, 

In [ ]:
print(df['sentimen'].value_counts())

1    22390
2    22060
0    15612
Name: sentimen, dtype: int64


In [ ]:
df.to_csv('databarulagi.csv',index = False)